In [2]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import gc 
#from spacy.tokenizer import Tokenizer
#from spacy.lang.en import English
import re

In [7]:
#hide
!pip install -Uqq fastbook
#hide
from fastbook import *
from IPython.display import display,HTML
from fastai.text.all import *
from fastai.text.all import Tokenizer as Tokenize

import fastbook
fastbook.setup_book()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.5 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.5 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
distributed 2021.5.0 requires dask==2021.05.0, but you have dask 2021.4.0 which is incompatible.


In [166]:
pd.options.display.max_colwidth = 150
df = pd.read_csv("df_tweet.csv")

# df.tweet = df.tweet.str.split(pat=': ').str[-1]
# df.tweet.to_csv("df_tweet.csv")

In [167]:
df = df[~df['tweet'].str.contains('http')].reset_index()
len(df)

19500

In [168]:
df['tweet'][22:24]

22    " fuck no that bitch dont even suck dick " &#128514;&#128514;&#128514; the Kermit videos bout to fuck IG up
23                                                                 " her pussy lips like Heaven doors " &#128524;
Name: tweet, dtype: object

In [175]:
test = df.tweet[51]
rnd_name = str(names.get_first_name())
matched = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), test)
matched
test_2 = 'How u gone bring ur side bitch to a game where &#128553;You know Ya gf fri&#128553;ends at ?! &#128553;&#128553;" I SWEAR!!!!!'
matched_2 = re.sub('&#.+?;', rnd_name+str(''), test_2)
matched_2

'How u gone bring ur side bitch to a game where DorothyYou know Ya gf friDorothyends at ?! DorothyDorothy" I SWEAR!!!!!'

In [180]:
import names

for i in range(len(df['tweet'])):
    rnd_name = str(names.get_first_name())
    df['tweet'][i]  = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), df['tweet'][i])
    df['tweet'][i]  = re.sub('&.+?;', '', df['tweet'][i])

<ipython-input-180-480d5481ee68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i]  = re.sub('@.+?(\b|\s|$)', rnd_name+str(' '), df['tweet'][i])
<ipython-input-180-480d5481ee68>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i]  = re.sub('&.+?;', '', df['tweet'][i])


In [184]:
df.to_csv("cleand_tweet.csv")

In [185]:
tokenizer = WordTokenizer()
tkn = Tokenize(tokenizer)

In [186]:
print(df.tweet[2])
print(coll_repr(tkn(df.tweet[2]), 30))

Alma she look like a tranny
(#8) ['xxbos','xxmaj','alma','she','look','like','a','tranny']


In [187]:
toks200 = df.tweet[:200].map(tkn)
print(coll_repr(toks200[0], 30))

(#18) ['xxbos','boy','dats','cold','…','tyga','dwn','bad','for','cuffin','dat','hoe','in','the','1st','place','!','!']


In [188]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,50)

'(#200) [\'xxunk\',\'xxpad\',\'xxbos\',\'xxeos\',\'xxfld\',\'xxrep\',\'xxwrep\',\'xxup\',\'xxmaj\',\'"\',\'i\',\'bitch\',\'.\',\'a\',\'you\',\'the\',\'that\',\'to\',\'!\',"n\'t",\'bitches\',\'hoes\',\'do\',\'my\',\'like\',\'me\',\',\',\'pussy\',\'be\',\'for\',\'?\',\'on\',\'up\',\'is\',\'in\',\'and\',\'it\',\'with\',\'get\',\'these\',\'if\',\'…\',"\'m",\'of\',\'shit\',\'..\',\'nt\',\'all\',\'fuck\',\'but\'...]'

In [189]:
nums200 = toks200.map(num)

In [190]:
dl = LMDataLoader(nums200)

In [191]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 57]), torch.Size([64, 57]))

In [192]:
## Checking
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxunk xxunk xxunk … xxunk xxunk bad for xxunk xxunk hoe in the xxunk xxunk ! ! xxbos xxmaj'

In [193]:
dls = TextDataLoaders.from_df(df, text_col='tweet', is_lm=True)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


C:\Users\tonyl\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [194]:

learn = language_model_learner(
    dls, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [195]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Ti'

In [196]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.876323,4.533437,0.224273,93.077942,00:11


In [ ]:
learn.save('10epoch')

In [ ]:
learn = learn.load('10epoch')

In [197]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.341064,4.406570,0.242332,81.987801,00:12
1,4.234272,4.334343,0.242510,76.274826,00:12
2,4.051903,4.294801,0.258337,73.317604,00:12
3,3.775156,4.349750,0.257440,77.459099,00:12
4,3.435732,4.463824,0.252952,86.818855,00:12
5,3.083234,4.642047,0.246090,103.756561,00:12
6,2.712221,4.816313,0.240571,123.508904,00:12
7,2.401290,4.968582,0.237575,143.822754,00:12
8,2.204347,5.044863,0.236154,155.223068,00:12
9,2.076193,5.075380,0.236039,160.032974,00:12


In [226]:
learn.save_encoder('finetuned')

In [224]:
TEXT = "My friend Dudu"
N_WORDS = 50
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [225]:
print("Inputted sentence: "+TEXT)
print("\nGenerated sentences...")
print("\n".join(preds))

Inputted sentence: My friend Dudu

Generated sentences...
My friend Xxunk is a bitch If Nick Cannon was nt supposed to get his ass beat in this fight he 's a lil bitch Eric Yankee games are so funny . Any bitch who did it in baseball is the ultimate bitch .
My friend Xxunk said " i wanna suck your dick when i take it from the bitches " … I m a act like a bitch My dad just told his dad to leave them crib bitches saying " i 'm a cold ass bitch " I 'm
